In [0]:
select 
  ws.website_session_id,
  min(wp.website_pageview_id) as first_pageview_id,
  count(wp.website_pageview_id) as count_pageviews
from website_sessions ws
left join website_pageviews wp
on 
  ws.website_session_id = wp.website_session_id
where 
  ws.created_at > '2012-08-05'
  and ws.created_at < '2012-09-05'
  and ws.utm_source = 'gsearch'
  and ws.utm_campaign = 'nonbrand'
group by 1

select 
* 
from website_pageviews as wp
where wp.pageview_url = '/lander-1' or wp.pageview_url = '/thank-you-for-your-order'

select 
* 
from website_pageviews as wp
where wp.website_session_id = 11729
order by 2

SELECT website_session_id
FROM website_pageviews
WHERE (website_session_id, created_at) IN (
        SELECT website_session_id, MIN(created_at)
        FROM website_pageviews
        GROUP BY website_session_id
    )
  AND pageview_url = '/lander-1'
  AND website_session_id IN (
        SELECT website_session_id
        FROM website_pageviews
        WHERE (website_session_id, created_at) IN (
                SELECT website_session_id, MAX(created_at)
                FROM website_pageviews
                GROUP BY website_session_id
            )
          AND pageview_url = '/thank-you-for-your-order'
    );

-------------------------------step 1 ------------------------------
select 
  ws.website_session_id,
  wp.pageview_url,
  wp.created_at as pageview_created_at,
  case when wp.pageview_url = '/lander-1' then 1 else 0 end as products_page,
  case when wp.pageview_url = '/products' then 1 else 0 end as products_page,
  case when wp.pageview_url = '/the-original-mr-fuzzy' then 1 else 0 end as myfuzzy_page,
  case when wp.pageview_url = '/cart' then 1 else 0 end as cart_page,
  case when wp.pageview_url = '/shipping' then 1 else 0 end as shipping_page,
  case when wp.pageview_url = '/billing' then 1 else 0 end as billing_page,
  case when wp.pageview_url = '/thank-you-for-your-order' then 1 else 0 end as thankyou_page
from website_sessions ws
left join website_pageviews wp
on 
  ws.website_session_id = wp.website_session_id
where 
  ws.created_at between '2012-08-05' and '2012-09-05'
  and wp.pageview_url in ('/lander-1', '/products', '/the-original-mr-fuzzy', '/cart', '/shipping', '/billing','/thank-you-for-your-order')
  and ws.utm_source = 'gsearch'
  and ws.utm_campaign = 'nonbrand'
order by 
  ws.website_session_id,
  wp.created_at

-------------------------------step 2 ------------------------------
create temporary table session_level_made_it_flags
select 
  website_session_id
  ,max(products_page) as product_made_it
  ,max(myfuzzy_page) as myfuzzy_made_it
  ,max(cart_page) as cart_made_it
  ,max(shipping_page) as shipping_made_it
  ,max(billing_page) as billing_made_it
  ,max(thankyou_page) as thankyou_made_it
  from (
  select 
      ws.website_session_id,
      wp.pageview_url,
      wp.created_at as pageview_created_at,
      case when wp.pageview_url = '/lander-1' then 1 else 0 end as lander_page,
      case when wp.pageview_url = '/products' then 1 else 0 end as products_page,
      case when wp.pageview_url = '/the-original-mr-fuzzy' then 1 else 0 end as myfuzzy_page,
      case when wp.pageview_url = '/cart' then 1 else 0 end as cart_page,
      case when wp.pageview_url = '/shipping' then 1 else 0 end as shipping_page,
      case when wp.pageview_url = '/billing' then 1 else 0 end as billing_page,
      case when wp.pageview_url = '/thank-you-for-your-order' then 1 else 0 end as thankyou_page
    from website_sessions ws
    left join website_pageviews wp
    on 
      ws.website_session_id = wp.website_session_id
    where 
      ws.created_at between '2012-08-05' and '2012-09-05'
      and wp.pageview_url in ('/lander-1', '/products', '/the-original-mr-fuzzy', '/cart', '/shipping', '/billing','/thank-you-for-your-order')
      and ws.utm_source = 'gsearch'
      and ws.utm_campaign = 'nonbrand'
    order by 
      ws.website_session_id,
      wp.created_at
  ) AS pageview_level
group by 1

-------------------------------step 3------------------------------
select 
  count(distinct website_session_id) as sessions,
  count(distinct case when product_made_it = 1 then website_session_id end) as products_page,
  count(distinct case when myfuzzy_made_it = 1 then website_session_id end) as myfuzzy_page,
  count(distinct case when cart_made_it = 1 then website_session_id end) as cart_page,
  count(distinct case when shipping_made_it = 1 then website_session_id end) as shipping_page,
  count(distinct case when billing_made_it = 1 then website_session_id end) as billing_page,
  count(distinct case when thankyou_made_it = 1 then website_session_id end) as thankyou_page
from 
  session_level_made_it_flags;

----------------- OR -----------------

-- select count(*)
-- from session_level_made_it_flags
-- where thankyou_made_it = 1 and billing_made_it = 1 and shipping_made_it = 1 and cart_made_it = 1 and myfuzzy_made_it = 1 and product_made_it = 1


-------------------------------step 4------------------------------
select 
  count(distinct website_session_id) as sessions,
  -- count(distinct case when product_made_it = 1 then website_session_id end) as products_page,
  round(count(distinct case when product_made_it = 1 then website_session_id end) / count(distinct website_session_id),2)  as clickthroughrate_products_page,

  -- count(distinct case when myfuzzy_made_it = 1 then website_session_id end) as myfuzzy_page,
  round(count(distinct case when myfuzzy_made_it = 1 then website_session_id end) / count(distinct website_session_id),2)  as clickthroughrate_mrfuzzy_page,

  -- count(distinct case when cart_made_it = 1 then website_session_id end) as cart_page,
  round(count(distinct case when  cart_made_it = 1 then website_session_id end) / count(distinct website_session_id),2)  as clickthroughrate_cart_page,

  -- count(distinct case when shipping_made_it = 1 then website_session_id end) as shipping_page,
  round(count(distinct case when shipping_made_it = 1 then website_session_id end) / count(distinct website_session_id),2)  as clickthroughrate_shipping_page,

  -- count(distinct case when billing_made_it = 1 then website_session_id end) as billing_page,
  round(count(distinct case when billing_made_it = 1 then website_session_id end) / count(distinct website_session_id),2)  as clickthroughrate_billing_page,

  -- count(distinct case when thankyou_made_it = 1 then website_session_id end) as thankyou_page,
  round(count(distinct case when thankyou_made_it = 1 then website_session_id end) / count(distinct website_session_id),2)  as clickthroughrate_thankyou_page
from 
  session_level_made_it_flags

-- website manager wants to focus on the lander, mrfuzzy and billing page 